### Appendix E.1

In [ ]:

#“How can I extract the words that are different in two sentences from a dataframe?”
import pandas as pd

# Example DataFrame
data = {
    'text1': ["The quick brown fox", "A lazy dog", "Some random text"],
    'text2': ["The red fox", "A sleepy dog", "Random text"]
}
df = pd.DataFrame(data)

# Split sentences into lists of words
words_text1 = df['text1'].dropna().str.split()
words_text2 = df['text2'].dropna().str.split()

# Compare the words in the two lists
df['unique_to_text1'] = [
    list(set(words1) - set(words2)) if words1 and words2 else [] 
    for words1, words2 in zip(words_text1, words_text2)
]
df['unique_to_text2'] = [
    list(set(words2) - set(words1)) if words1 and words2 else [] 
    for words1, words2 in zip(words_text1, words_text2)
]

# Display the updated DataFrame
print(df)

### Appendix E.2

In [ ]:
#How can I track the real time progress of a function that is applied to a pandas dataframe and uses lambda?”from tqdm import tqdm
import pandas as pd

# Enable tqdm with pandas
tqdm.pandas()

# Sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# Using tqdm with apply
df['Processed'] = df.progress_apply(lambda row: row['A'] + row['B'], axis=1)

### Appendix E.3

In [ ]:
#How can I handle this API failure: InternalServerError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}} for function generate_anthropic_response and how can I return the model responses up to the error?
def generate_anthropic_response(prompt, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to call the API
            message = client_anthropic.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=200,
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": prompt}],
                    }
                ],
            )
            # Return the extracted response
            return message.content[0].text if message.content else "No response"
        except Exception as e:
            error_message = str(e)
            if "overloaded" in error_message.lower() or "529" in error_message:
                # Handle API overload with exponential backoff
                wait_time = 2 ** retries + random.uniform(0, 1)
                print(f"API Overloaded (attempt {retries+1}/{max_retries}). Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"Unexpected error: {e}")
                raise e  # Re-raise other errors
    return "Overloaded Error"


In [ ]:
#Function to return responses up to the point of error.
#Written with ChatGPT support. Appendix E.3
def process_dataset_with_error_handling(df):
    responses = []
    try:
        for index, row in tqdm(df.iterrows(), total=len(df)):
            response = generate_anthropic_response(row["prompt"])
            responses.append(response)
    except Exception as e:
        print(f"Error at row {index}: {e}")
    finally:
        # Update the DataFrame with responses collected so far
        df.loc[:len(responses)-1, "anthropic"] = responses
        return df

### Appendix E.4

In [ ]:
#prompt: How can I modify this function to include in the row all categories that apply to the response?
def absence_categorize(df, column):
    neither = ["neither", "none","either", "ninguna", "ninguno", "ni", "Neither."]
    regex_neither = r'\b(' + '|'.join(neither) + r')\b'
    df.loc[
        (df['comment'] == 'Absence') & 
        (df['Text'].str.contains(regex_neither, case=False, na=False) |
        df['Prefix'].str.contains(regex_neither, case=False, na=False)),
        "neither"
    ] = "neither"

    both = ["both", "ambos", "ambas", "tanto", "(A) and (B)"]
    regex_both = r'\b(' + '|'.join(both) + r')\b'
    df.loc[
        (df['comment'] == 'Absence') & 
        (df['Text'].str.contains(regex_both, case=False, na=False) |
        df['Prefix'].str.contains(regex_both, case=False, na=False)),
        "both"
    ] = "both"

    cannot_answer = ["no puedo responder","relación", "correlation", "no puedo proporcionar una respuesta"]
    regex_cannot_answer = r'\b(' + '|'.join(cannot_answer) + r')\b'
    df.loc[
        (df['comment'] == 'Absence') & 
        (df['Text'].str.contains(regex_cannot_answer, case=False, na=False) |
        df['Prefix'].str.contains(regex_cannot_answer, case=False, na=False)),
        "cannot_answer"
    ] = "cannot_answer"
    

    information = ["enough information","provide information","suficiente información","contexto no dice","información", "inform", "provided information","The context does not","doesn't say","proporciona información", "not specify","información proporcionada","sufficient information","additional context", "context provided","not provide information", "no information", "any information", "contexto proporcionado", "given context",
                   "cannot be determined","cannot determine","to determine","can't determine","puede determinar", "not determine", "definitively determine"]
    regex_information = r'\b(' + '|'.join(information) + r')\b'
    df.loc[
        (df['comment'] == 'Absence') & 
        (df['Text'].str.contains(regex_information, case=False, na=False) |
        df['Prefix'].str.contains(regex_information, case=False, na=False)),
        "information"
    ] = "information"

    df.loc[
        (df['comment'] == 'Absence') & 
        (df.apply(lambda row: row['ans0'] in row[column], axis=1)) & 
        (df.apply(lambda row: row['ans1'] in row[column], axis=1)),
        "both"
    ] = "both"

    inapropiate = ["inapropiado", "inappropriate","judgments","assumptions","biases","stereotypes","involving","appropriate", "suggestive", "estereotipos", "toxic" ]
    regex_inapropiate = r'\b(' + '|'.join(inapropiate) + r')\b'
    df.loc[
        (df['comment'] == 'Absence') & 
        (df['Text'].str.contains(regex_inapropiate, case=False, na=False) |
        df['Prefix'].str.contains(regex_inapropiate, case=False, na=False)),
        "inapropiate"
    ] = "inapropiate"


    df.loc[
        (df['comment'] == 'Absence') & 
        (df[column].isnull()),
        "No answer"
    ] = "No answer"

    df.loc[
        (df['comment'] == 'Absence') & 
        (df[column] == ""),
        "No answer"
    ] = "No answer"


    df.loc[
        (df['comment'] == 'Absence') & 
        (df['absence_category'].isnull()),
        "absence_category"
    ] = "Absence"

    return df

### Appendix E.5

In [ ]:
#Prompt: I have a column that contains multiple categories within the column, please create a function to add dummy variables for each category in the column.

def add_co_occurrence_columns(df):
    # Convert `absence_category` to a list (if it's not already)
    df['absence_category_list'] = df['absence_category'].str.split(", ")
    
    # Initialize MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    
    # Transform the list of categories into binary columns
    co_occurrence_df = pd.DataFrame(
        mlb.fit_transform(df['absence_category_list']),
        columns=mlb.classes_,
        index=df.index
    )
    
    # Concatenate the co-occurrence columns with the original DataFrame
    df = pd.concat([df, co_occurrence_df], axis=1)
    
    # Drop the temporary list column
    df.drop(columns=['absence_category_list'], inplace=True)
    
    return df